In [1]:
from IPython import get_ipython

%matplotlib qt
%load_ext autoreload
%autoreload 2


import time, os, sys, shutil
from utils.fitting_utils import *

# for math and plotting
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Qt5Agg')
# %matplotlib notebook
# %matplotlib widget
# %matplotlib qt5
# %matplotlib nbagg

from itertools import compress # for list selection with logical
from tqdm import tqdm

from multiprocessing import Process

# ALLSO JIT STUFF
from numba import jit, njit

# and pytorch
import torch

import sys, os, pickle
# import cv2
from colour import Color
import h5py
from tqdm import tqdm, tqdm_notebook
import glob
import itertools

In [2]:
# Check CUDA
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
torch_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# torch_device = "cpu"

print(torch_device)

True
1
GeForce RTX 2080 Ti
cuda:0


In [3]:
# setup for pretty plotting
matplotlib.rcParams['font.sans-serif'] = "Liberation Sans"
# Then, "ALWAYS use sans-serif fonts"
matplotlib.rcParams['font.family'] = "sans-serif"

matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 13})

from palettable.cmocean.sequential import Algae_6
cmpl = Algae_6.mpl_colors

def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 10))  # outward by 10 points
            spine.set_smart_bounds(True)
        else:
            spine.set_color('none')  # don't draw spine

    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])

    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])


# Load the pre-processed data and display an example frame

In [4]:
data_folder = 'example_data/tracking/'

top_folder_0 = '/media/chrelli/Data0/recording_20200821-131033'
top_folder_1 = '/media/chrelli/Data1/recording_20200821-131033'

# validation dataset with LASER ON 90 fps
top_folder_0 = '/media/chrelli/Data0/recording_20200828-114251'
top_folder_1 = '/media/chrelli/Data1/recording_20200828-114251'

# Data with female partner 3500 exposure
top_folder_0 = '/media/chrelli/Data0/recording_20201110-102009/'
top_folder_1 = '/media/chrelli/Data1/recording_20201110-102009/'

top_folder_0 = '/media/chrelli/SSD4TB/Data0_backup/recording_20201110-102009/'
top_folder_1 = '/media/chrelli/SSD4TB/Data1_backup/recording_20201110-102009/'


# # Data with male partner 3500 exposure
# top_folder_0 = '/media/chrelli/Data0/recording_20201110-105540/'
# top_folder_1 = '/media/chrelli/Data1/recording_20201110-105540/'
# top_folder_0 = '/media/chrelli/SSD4TB/Data0_backup/recording_20201110-105540'
# top_folder_1 = '/media/chrelli/SSD4TB/Data1_backup/recording_20201110-105540'



# # Thrusting data
# top_folder_0 = '/media/chrelli/Data0/recording_20201112-104816/'
# top_folder_1 = '/media/chrelli/Data1/recording_20201112-104816/'



data_folder = top_folder_0


# load ALL the frames as jagged lines 
with h5py.File(data_folder+'/pre_processed_frames.hdf5', mode='r') as hdf5_file:
    print(hdf5_file.keys())
    print(len(hdf5_file['dataset']))
    jagged_lines = hdf5_file['dataset'][...] 


<KeysViewHDF5 ['dataset']>
73123


In [5]:
from utils.cuda_tracking_utils import unpack_from_jagged, cheap4d
# kill first 6 secs of the frames (delay is ~180)
start_frame = 10*60
pos, pos_weights, keyp, pkeyp, ikeyp = unpack_from_jagged(jagged_lines[start_frame])
print(ikeyp)
print(pos.shape)
fig = plt.gcf()
plt.title("N positions is {}".format(pos.shape))
plt.show()
cheap4d(pos,keyp,ikeyp)


[1 1 3 1 1 2 1 1 2 3 2]
(3163, 3)


In [6]:
# AUTO-start the tracking, start with frame 0, and loop until there is a frame, where the animals are reasonably far apart!

plt.close('all')

from utils.cuda_tracking_utils_weights_for_figures import body_constants, particles_to_distance_cuda, clean_keyp_by_r
from utils.cuda_tracking_utils_weights_for_figures import loading_wrapper
from utils.clicking import *

from scipy.spatial.distance import pdist, squareform
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.cluster.vq import vq, kmeans, whiten

def bimodality_coeff(dat):
    # from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3791391/
    m3 = skew(dat)
    m4 = kurtosis(dat)
    n = len(dat)
    BC = (m3**2 + 1)/(m4**2 + 3 * ((n-1)**2)/( (n-2)*(n-3) ) )
    return BC

def check_mouse_separation(keyp,ikeyp, has_implant = False):
    # get the xy-coordinates of the keypoints
    xy_head = keyp[(ikeyp == 1)| (ikeyp == 2) ,:].cpu().numpy()
    xy_tail = keyp[ikeyp == 3,:].cpu().numpy()
    
    if len(xy_head) < 2 or len(xy_tail) < 2:
        return False, np.nan*np.ones((2,3)), np.nan*np.ones((2,3))
    
    # kmeans two clusters # a bit slow, but w/e
    c_head,distortion_head = kmeans(xy_head, 2)
    c_tail,distortion_tail = kmeans(xy_tail, 2)
    
    # associate to partners, since there are only two, we can do it this way
    match_0 = np.argmin(np.sum((c_tail - c_head[0,:])**2,1))
    match_1 = np.argmin(np.sum((c_tail - c_head[1,:])**2,1))
    
    # assemble the mice
    mouse_0 = np.vstack((c_head[0,:],c_tail[match_0,:]))
    mouse_1 = np.vstack((c_head[1,:],c_tail[match_1,:]))
    
    # check that all the cross-mouse distances are larger than a threshold
    # h2h, t2t, and the two h2t
    cross_difference = mouse_0[[0,1,0,1],:] - mouse_1[[0,1,1,0],:] 
    cross_dist = np.sqrt( np.sum(cross_difference**2,1) )
    
    # Hmm
    separation_cutoff = 0.05 # let's do 7 cm!
    sep_criterion = np.all(cross_dist > separation_cutoff)
    
    #also make sure the two mice are long enough!
    mouse_lengths = np.array([np.linalg.norm(np.diff(mouse_0,axis = 0)), 
                              np.linalg.norm(np.diff(mouse_1,axis = 0))])
    length_cutoff = 0.05 # let's do 7 cm!
    l_criterion = np.all(mouse_lengths > length_cutoff)

    criterion = sep_criterion * l_criterion
    
    # If we are looking for an implant, we also want to make sure that there are visible implant keypoints
    if has_implant:
        impl_criterion = np.sum(ikeyp.cpu().numpy() == 0) > 0
        criterion = criterion * impl_criterion
        
        # make sure that mouse0 has the implant!
        xy_impl = keyp[ikeyp == 0,:].cpu().numpy()
        c_impl = np.mean(xy_impl,axis=0)
        
        # distance to mice, sorting by the head, see above
        mice= [mouse_0,mouse_1]
        d1 = np.sqrt(np.sum( (c_head - c_impl)**2 ,axis = 1))

        # since there are only two, we can just do logic, little bit hacky 
        implanted_index = np.argmin(d1).astype('bool')
        mouse_0 = np.vstack([mice[implanted_index.astype(int)],c_impl])
        mouse_1 = mice[ (~implanted_index).astype(int) ]
        
    return criterion, mouse_0, mouse_1

def plot_top_view(pos,keyp,ikeyp,mouse_0=None,mouse_1=None):
    keyp_colors = ['dodgerblue','green','red','orange']
    plt.figure()

    posi = pos.cpu().numpy()
    plt.plot(posi[:,0],posi[:,1],'.',alpha=.1,c='k')

    if mouse_0 is not None:
        lw = 3
        plt.plot(mouse_0[:,0],mouse_0[:,1],':',lw=lw,c='k')
        plt.plot(mouse_1[:,0],mouse_1[:,1],':',lw=lw,c='peru')

        ss = 10
        plt.plot(mouse_0[0,0],mouse_0[0,1],'v',markersize=ss,c='k')
        plt.plot(mouse_1[0,0],mouse_1[0,1],'v',markersize=ss,c='peru')

        plt.plot(mouse_0[1,0],mouse_0[1,1],'o',markersize=ss,c='k')
        plt.plot(mouse_1[1,0],mouse_1[1,1],'o',markersize=ss,c='peru')

    for ik,colors in enumerate(keyp_colors):
        xy = keyp[ikeyp == ik,:2].cpu().numpy()
        xy = keyp[ikeyp == ik,:2].cpu().numpy()
        x = xy[:,0]
        y = xy[:,1]

        plt.plot(x,y,'o',c=keyp_colors[ik])

    plt.show()

def plot_3_views(pos,keyp,ikeyp,mouse_0=None,mouse_1=None,has_implant=False,savepath=None):
    keyp_colors = ['dodgerblue','green','red','orange']
    plt.figure(figsize = (9,6))

    for i_row in range(2):
        for i_sub,(p1,p2) in enumerate(zip([0,0,1],[1,2,2])):
            plt.subplot(2,3,1+i_sub+3*i_row)


            if i_row ==0:
                posi = pos.cpu().numpy()
                plt.plot(posi[:,p1],posi[:,p2],'.',alpha=.01,c='k')

                for ik,colors in enumerate(keyp_colors):
                    xy = keyp[ikeyp == ik,:].cpu().numpy()
                    xy = keyp[ikeyp == ik,:].cpu().numpy()
                    x = xy[:,p1]
                    y = xy[:,p2]

                    plt.plot(x,y,'o',c=keyp_colors[ik],alpha = 0.3)


            if mouse_0 is not None:
                lw = 1.5
                mz = 2
                if mouse_0.shape[0] > 2:
                    plt.plot(mouse_0[[2,0,1],p1],mouse_0[[2,0,1],p2],'o-',markersize=mz,lw=lw,c='k')
                else:
                    plt.plot(mouse_0[:,p1],mouse_0[:,p2],'o-',markersize=mz,lw=lw,c='k')
                plt.plot(mouse_1[:,p1],mouse_1[:,p2],'o-',markersize=mz,lw=lw,c='peru')
                    
            plt.xlim([-.2,.2])
            plt.ylim([-.2,.2])
            
    if savepath is not None:
        plt.savefig(savepath,dpi=300)

    plt.show()    

def generate_starting_x0(pos,keyp,ikeyp,mouse_0,mouse_1, has_implant = False):

    # x0_start is a,b,s, theta, phi, xyz
    # negative angle on b, because the rotation is down
    vec_0 = mouse_0[0,:] - mouse_0[1,:]
    center_0 = mouse_0[1,:] + 0.5 *vec_0
    # gamma and beta:
    g_0 = angle_between(np.array([1,0,0]),vec_0 *np.array([1,1,0]) )
    b_0 = - angle_between(vec_0 * np.array([1,1,0]),vec_0) 

    # x0_start is a,b,s, theta, phi, xyz
    vec_1 = mouse_1[0,:] - mouse_1[1,:]
    center_1 = mouse_1[1,:] + 0.5 *vec_1
    g_1 = angle_between(np.array([1,0,0]),vec_1 *np.array([1,1,0]) )
    b_1 = - angle_between(vec_1 * np.array([1,1,0]),vec_1)  


    if has_implant: # if there is an implant, we just squeeze it in to the right place!
        mouse_0_start = np.hstack([ np.array([b_0,g_0,.9,0.,0.,0.]), center_0])
    else:
        mouse_0_start = np.hstack([ np.array([b_0,g_0,.9,0.,0.]), center_0])

    mouse_1_start = np.hstack([ np.array([b_1,g_1,.9,0.,0.]), center_1])        
    x0_start = np.hstack([mouse_0_start, mouse_1_start])
    
    return x0_start

has_implant = True

# skip first 5 secs to make sure all cameras are going:
for start_frame in np.arange(5*60,30000,100):
    pos,pos_weights,keyp,ikeyp = loading_wrapper(start_frame,jagged_lines)
    
    criterion, mouse_0, mouse_1 = check_mouse_separation(keyp,ikeyp,has_implant = has_implant)
    
    if criterion:
#         plot_top_view(pos,keyp,ikeyp,mouse_0,mouse_1)
        print("Found a good starting point on frame # {} !".format(start_frame) )
    
        # # convert the mouse_0, mouse_1 into a starting guess
        x0_start = generate_starting_x0(pos,keyp,ikeyp,mouse_0,mouse_1, has_implant = has_implant)

        plot_3_views(pos,keyp,ikeyp,mouse_0,mouse_1,has_implant=has_implant,savepath='../st')          
        
        break
        


Found a good starting point on frame # 1400 !


/home/chrelli/anaconda2/envs/mousepose/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/chrelli/anaconda2/envs/mousepose/lib/python3.6/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [8]:
pos,pos_weights,keyp,ikeyp = loading_wrapper(start_frame,jagged_lines)

part = torch.Tensor(x0_start).to(torch_device).unsqueeze(0)
# no need for the the particle to have gradients
part.requires_grad = False

print(part)
print(part.shape)
print(pos.shape)

tensor([[-0.4757,  1.3763,  0.9000,  0.0000,  0.0000,  0.0000, -0.1049,  0.0409,
          0.0219,  0.3951, -0.1016,  0.9000,  0.0000,  0.0000, -0.0779, -0.0545,
          0.0239]], device='cuda:0')
torch.Size([1, 17])
torch.Size([3854, 3])


# Import the actual particle filter tracking engine, 'MousePFilt', and fit the first frame

In [9]:
# get the limits for the tracking and the residual functions
from utils.cuda_tracking_utils_weights_for_figures import search_cone, global_min, global_max
from utils.cuda_tracking_utils_weights_for_figures import add_implant_residual,add_body_residual,add_ass_residual, add_ear_residual, add_nose_residual

# for single mice
# global_min = global_min[:3,4:]
# global_max = global_max[:,:3]
from utils.cuda_tracking_utils_weights_for_figures import search_cone_noimp, global_min_noimp, global_max_noimp

print(global_max_noimp)
print(global_min_noimp)

print(global_max)
print(global_min)

tensor([[   inf,    inf, 1.0000, 1.0467, 1.0467,    inf,    inf, 0.0700,    inf,
            inf, 1.0000, 1.0467, 1.0467,    inf,    inf, 0.0700]],
       device='cuda:0')
tensor([[   -inf,    -inf,  0.3000, -1.0467, -1.0467,    -inf,    -inf,  0.0000,
            -inf,    -inf,  0.3000, -1.0467, -1.0467,    -inf,    -inf,  0.0000]],
       device='cuda:0')
tensor([[   inf,    inf, 1.0000,    inf, 1.0467, 1.0467,    inf,    inf, 0.0700,
            inf,    inf, 1.0000, 1.0467, 1.0467,    inf,    inf, 0.0700]],
       device='cuda:0')
tensor([[   -inf,    -inf,  0.3000,    -inf, -1.0467, -1.0467,    -inf,    -inf,
          0.0000,    -inf,    -inf,  0.3000, -1.0467, -1.0467,    -inf,    -inf,
          0.0000]], device='cuda:0')


In [10]:
from utils.cuda_tracking_utils_weights_for_figures import MousePFilt, make_some_bounds,particles_to_body_supports_cuda

if has_implant:
    upper_bound,lower_bound = make_some_bounds(part,search_cone/3,global_max,global_min)
    pzo = MousePFilt(swarm_size = 150)
else:
    part_noimp = part
    upper_bound,lower_bound = make_some_bounds(part_noimp,search_cone_noimp/3,global_max_noimp,global_min_noimp)    
    pzo = MousePFilt(swarm_size = 150,has_implant = False) # fix

pzo.search_space(upper_bound,lower_bound)

# populate the tracker
pzo.populate(sobol = True)

# send the data for tracking
pzo.pos = pos[::1,:]
pzo.pos_weights = pos_weights
pzo.keyp = keyp
pzo.ikeyp = ikeyp

pzo.max_iterations = 1
self = pzo
pzo.run2(cinema=False, fast_sort = False)
plt.close('all')
print(np.mean(self.time_benchmarking))
# if has_implant:
#     pzo.plot_status(reduce_mean=True,keep_open=True,plot_ellipsoids=True)
# else:
#     pzo.plot_status_noimpl(reduce_mean=False,keep_open=True,plot_ellipsoids=True)

it 0 of 1, best loss is 1987.0142822265625, time 0.03353850399980729
0.03353850399980729


In [35]:
# 
plt.close('all')
swarm_sizes = [10,50,100,200,400,600]
gpu_fast_sort = [15.2,15.4,17.8,21.3,35.0,51.5]
gpu_normal_sort = [14.7,15.3,17.1,20.8,34.6,50]


plt.figure(figsize = (2.5,2))
# plt.plot(swarm_sizes,gpu_fast_sort,'--o', label = 'GPU, topk')
plt.plot(swarm_sizes,gpu_normal_sort,'-ok', label = 'GPU, sort')
# swarm_sizes = [10,50,100,150,200,400,600]
# cpu_fast_sort = [23.6, 49.3, 117.8, 177.8, 266.8, 12.8, 1654.0]
# cpu_normal_sort = [24.8, 48.3, 116.5, 181.7, 280.3, 816.7, 1658.6]

swarm_sizes = [10,50,100,200,400,600]
cpu_fast_sort = [23.6, 49.3, 117.8, 266.8, 812.8, 1654.0]
cpu_normal_sort = [24.8, 48.3, 116.5, 280.3, 816.7, 1658.6]


# plt.plot(swarm_sizes,cpu_fast_sort,'--o', label = 'CPU, topk')
plt.plot(swarm_sizes,cpu_normal_sort,'--ok', label = 'CPU, sort')
# plt.legend()
# plt.xlabel("Particles")
# plt.ylabel("Iteration time [ms]")
plt.xticks(swarm_sizes,rotation=90)
ax = plt.gca()
# ax.set_xticklabels('')

ax = plt.gca()
# ax.set_ylim([0, None])
plt.yscale('log')
plt.yticks([10,100,1000])
ax.set_yticklabels(['10 ms','100 ms','1 s'
                   ])

ax.set_yticklabels([])
ax.set_xticklabels([])

# plt.tight_layout()

plt.savefig('revision_figures/profile_network/GPUspeed.pdf',transparent= True)

plt.show()

In [30]:
plt.close('all')

# Make a wrapper to run the particle filter across all frames, set options

In [27]:

if has_implant:
    pzo = MousePFilt(swarm_size = 200)
    def pzo_wrapper(part,pos,pos_weights,keyp,ikeyp,pzo):
        upper_bound,lower_bound = make_some_bounds(part,search_cone/3,global_max,global_min)

        pzo.search_space(upper_bound,lower_bound)
        pzo.populate(sobol = True)

        pzo.pos = pos
        pzo.pos_weights = pos_weights

        pzo.keyp = keyp
        pzo.ikeyp = ikeyp

        pzo.max_iterations = 3

        pzo.run2(verbose=False,use_weights = False,barrier = True,fast_sort = True)

        return pzo.meanwinner
else:
    pzo = MousePFilt(swarm_size = 200,has_implant = False) # fix
    def pzo_wrapper(part,pos,pos_weights,keyp,ikeyp,pzo):
        upper_bound,lower_bound = make_some_bounds(part,search_cone_noimp/3,global_max_noimp,global_min_noimp)    

        pzo.search_space(upper_bound,lower_bound)
        pzo.populate(sobol = True)

        pzo.pos = pos
        pzo.pos_weights = pos_weights

        pzo.keyp = keyp
        pzo.ikeyp = ikeyp

        pzo.max_iterations = 3

        pzo.run2(verbose=False,use_weights = False,barrier = True,fast_sort = True)

        return pzo.meanwinner


# Make a function to dump plots during tracking

In [28]:
plt.close('all')

from utils.plotting_during_tracking import *

def plot_single_frame(part,pos, keyp, ikeyp,frame):
    plt.ioff()
    plt.close('all')
    # the winning mouse is the one, with the lowest final loss
    #end_loss = [np.mean(ll[-1:]) for ll in ll_holder]
    dist0,_,body_support_0 = particles_to_distance_cuda(part[:,:8],pos,implant = False)
    dist1,_,body_support_1 = particles_to_distance_cuda(part[:,8:],pos,implant = False)
    body_supports = [body_support_0,body_support_1]
    
    #best_idx = np.argmin(end_loss)
    #best_mouse = best_holder[best_idx]

    fig = plt.figure(figsize=(7.5,7.5))

    ax = fig.add_subplot(1, 1, 1, projection='3d')
    plot_particles_new_nose(ax,part.cpu().numpy(),pos.cpu().numpy(),body_constants,alpha = .5,keyp = keyp.cpu(), ikeyp = ikeyp.cpu(),body_supports = [ [i.cpu() for i in j] for j in body_supports] )
    
    plt.axis('tight')
    
    ax.set_xlim(-.10,.20)
    ax.set_ylim(-.20,.1)
    ax.set_zlim(0,.3)
    

    ax.view_init(elev=60., azim=-147.)

    
    plt.savefig('frames/temp/frame_'+str(frame).zfill(6)+'.png')
#     plt.show()
    
    plt.close('all')
    
# frame = start_frame
# plot_single_frame(part,pos, keyp, ikeyp,frame)

# And import a bank for online filtering and prediction

In [29]:
plt.close('all')

from utils.cuda_tracking_utils import rls_bank

def ML_predict(bank,i_frame,embedding,tracking_holder,guessing_holder):
    # # do the RLS step to predict the next step
    if (i_frame > embedding + 2)*True:
        x_train = np.flip( tracking_holder[:-1,(i_frame-embedding):i_frame],axis = 1)
        y_train = tracking_holder[:-1,i_frame]
        d = torch.from_numpy(y_train.copy())
        x = torch.from_numpy(x_train.copy())
        # make sure the type is right
        d = torch.tensor(d,dtype = torch.float32)
        x = torch.tensor(x,dtype = torch.float32)

        # and send to the holder
        bank.adapt(d,x)

        # guess the upcoming step!
        x_predict = torch.cat((d.unsqueeze(1),x[:,:-1]),1)
        
        part_guess = bank.predict(x_predict)
        if ( i_frame +1 ) < ( guessing_holder.shape[1] - 2 ):
            guessing_holder[:-1,i_frame+1] = part_guess[:].numpy()
        return bank,part_guess.unsqueeze(0),guessing_holder
    else:
        return bank,0.,guessing_holder
    

# Now, run the tracking across all frames and save to disk

In [30]:
# start_frame =  10*60

n_frames = len(jagged_lines)-1-start_frame

# do 1000 frames!
# n_frames = 1000
# do one min for profiling
# n_frames = 1*30*60 

end_frame = start_frame + n_frames

if has_implant:
    part = torch.Tensor(x0_start).to(torch_device).unsqueeze(0)
    pzo = MousePFilt(swarm_size = 200)
    pzo.has_implant = True
    part = pzo_wrapper(part,pos,pos_weights,keyp,ikeyp,pzo)

else:
    part = torch.Tensor(x0_start).to(torch_device).unsqueeze(0)   
    pzo = MousePFilt(swarm_size = 150,has_implant = False)
    pzo.has_implant = False
    part = pzo_wrapper(part,pos,pos_weights,keyp,ikeyp,pzo)

# for naming the tracked behavior    
if has_implant:
    var_names = ['b','c','s','psi','theta','phi','x','y','z','b','c','s','theta','phi','x','y','z','frame']
    ivar_names = ['b0','c0','s0','psi0','theta0','phi0','x0','y0','z0','b1','c1','s1','theta1','phi1','x1','y1','z1','frame']
else:
    var_names = ['b','c','s','theta','phi','x','y','z','b','c','s','theta','phi','x','y','z','frame']
    ivar_names = ['b0','c0','s0','theta0','phi0','x0','y0','z0','b1','c1','s1','theta1','phi1','x1','y1','z1','frame']
    
    

embedding = 5
bank = rls_bank(n_vars = part.shape[1], embedding=embedding)
bank.mu = .99

x0_trace = []
frame_trace = []
history_trace = []
# just make a numpy holder for it directly
# and a frame index which tells us which frame we're currently tracking
tracking_holder = np.zeros((part.shape[1]+1,n_frames))
guessing_holder = np.zeros((part.shape[1]+1,n_frames))*np.nan

from tqdm import tqdm, tqdm_notebook

with torch.no_grad():
    for i_frame, this_frame in enumerate(tqdm(range(start_frame,start_frame+n_frames))):
        # if we've learned, preditc

        # load and fit
        pos,pos_weights,keyp,ikeyp = pos,pos_weights,keyp,ikeyp = loading_wrapper(this_frame,jagged_lines)
        # optional, cut down the cloud a bit
#         pos = pos[::3,:]

#         keyp,ikeyp = clean_keyp_by_r(part,keyp,ikeyp,has_implant=has_implant)
    #     part,history = klm_routine(part,pos,keyp,ikeyp,max_iters = 100,verbose=False,save_history = True,ftol = 1e-4)
    #     part, histo = pzo_step(part,pos,keyp,ikeyp)
        part = pzo_wrapper(part,pos,pos_weights,keyp,ikeyp,pzo)
        # 3. add to fitting history
        x0_trace.append(part.cpu().numpy())
        frame_trace.append(this_frame)
    #     history_trace.append(history)
        # and update the frame index and the tracking_holder
        tracking_holder[:-1,i_frame] = part[0,:].cpu().numpy()
        tracking_holder[-1,i_frame] = this_frame

        # always adapt!
        if True:
            bank,part_guess,guessing_holder = ML_predict(bank,i_frame,embedding,tracking_holder,guessing_holder)

        if i_frame > 150 and True:
            # do prediction after the first 150 frames
            pass
    #         part_guess[:,[5,13]] = part[:,[5,13]]
    #         part = part_guess
    #         part[:,[0,1,2,6,7,8,9,10,11,14,15,16]] = part_guess[:,[0,1,2,6,7,8,9,10,11,14,15,16]]
            if has_implant:
                part[:,[6,7,8,14,15,16]] = part_guess[:,[6,7,8,14,15,16]].to(torch_device)
#                 part = part_guess.to(torch_device)
            else:
#                 part[:,[0,1,2,5,6,7,8,9,10,13,14,15]] = part_guess[:,[0,1,2,5,6,7,8,9,10,13,14,15]].to(torch_device)
                #  part = part_guess.to(torch_device)
                part[:,[5,6,7,13,14,15]] = part_guess[:,[5,6,7,13,14,15]].to(torch_device)
    
    
        if i_frame%2 == 0 and False:
            # fully update the 
            if i_frame > 150:
                plot_single_frame(part_guess.to(torch_device),pos, keyp, ikeyp,this_frame)    
            else:
                plot_single_frame(part,pos, keyp, ikeyp,this_frame)    

        if i_frame%500 == 0:
            top_folder = 'frames/'
            print("saving tracking at frame {} of {}...".format(i_frame,start_frame+n_frames))
            np.save(top_folder+'tracking_holder.npy',tracking_holder)
            np.save(top_folder+'guessing_holder.npy',guessing_holder)
            np.save(top_folder+'body_constants.npy',body_constants)
            print("tracking saved!")    
            
            tracked_behavior = {
                "var": var_names,
                "ivar": ivar_names,
                "body_constants": body_constants,
                "body_constant_names" : ["body_scale","a_hip_min","a_hip_max","b_hip_min","b_hip_max","a_nose","b_nose","d_nose","x_impl","z_impl","r_impl"],
                "start_frame": start_frame,
                "end_frame": this_frame, # this saves it druring running!
                "has_implant": has_implant,
                "tracking_holder": tracking_holder,
                "guessing_holder": guessing_holder,
                "data_folder": data_folder
                }

            print("pickling tracking at frame {}...".format(i_frame))
            with open(data_folder +'/tracked_behavior_in_progress.pkl', 'wb+') as f:
                pickle.dump(tracked_behavior,f)
            print("behavior tracking pickled!")     

            
# TODO also add the date of the folder as a string?


tracked_behavior = {
    "var": var_names,
    "ivar": ivar_names,
    "body_constants": body_constants,
    "body_constant_names" : ["body_scale","a_hip_min","a_hip_max","b_hip_min","b_hip_max","a_nose","b_nose","d_nose","x_impl","z_impl","r_impl"],
    "start_frame": start_frame,
    "end_frame": end_frame,
    "has_implant": has_implant,
    "tracking_holder": tracking_holder,
    "guessing_holder": guessing_holder,
    "data_folder": data_folder
}

print("pickling tracking at frame {}...".format(i_frame))
with open(data_folder +'/tracked_behavior.pkl', 'wb+') as f:
    pickle.dump(tracked_behavior,f)
print("behavior tracking pickled!")                

  0%|          | 1/71722 [00:00<2:39:30,  7.49it/s]

saving tracking at frame 0 of 73122...
tracking saved!
pickling tracking at frame 0...
behavior tracking pickled!


  0%|          | 8/71722 [00:00<2:01:49,  9.81it/s]/home/chrelli/anaconda2/envs/mousepose/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/chrelli/anaconda2/envs/mousepose/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
  1%|          | 502/71722 [00:47<1:59:14,  9.95it/s]

saving tracking at frame 500 of 73122...
tracking saved!
pickling tracking at frame 500...
behavior tracking pickled!


  1%|▏         | 1003/71722 [01:34<2:15:34,  8.69it/s]

...

saving tracking at frame 71000 of 73122...
tracking saved!
pickling tracking at frame 71000...
behavior tracking pickled!


100%|█████████▉| 71502/71722 [1:49:20<00:22,  9.97it/s]

saving tracking at frame 71500 of 73122...
tracking saved!
pickling tracking at frame 71500...
behavior tracking pickled!


100%|██████████| 71722/71722 [1:49:40<00:00, 10.90it/s]

pickling tracking at frame 71721...
behavior tracking pickled!


In [15]:
#%% Plot tracked data to see that everything is fine
plt.close('all')
plt.figure()

if has_implant:
    NNN = ['b','c','s','psi','theta','phi','x','y','z','b','c','s','theta','phi','x','y','z']
else:
    NNN = ['b','c','s','theta','phi','x','y','z','b','c','s','theta','phi','x','y','z']
    
for ii,name in enumerate(NNN):
    plt.subplot(len(NNN),1,ii+1)
    index = np.arange(tracking_holder.shape[1])
    plt.plot(index[:i_frame],tracking_holder[ii,:i_frame])
    plt.plot(index[:i_frame],guessing_holder[ii,:i_frame])
    plt.ylabel(str(ii)+'_'+name)
plt.show()
 

In [16]:
# TRY to develop a kind of 3D kalman better version of the KRLS-T
# TRY to penalize if the center of the implant is lower than the neck of the mouse
# TRY to wrap it for Bonsai
# TRY to make a much more lightweight GUI for playing the data as a video. Preferably for a 
# jupyter notebook
# Add tracking time
# Make nice GUI
# ENsure that the TTL pulses are recorded 
# Penalize implant to the side of head
# Penalize flying tail